<a href="https://colab.research.google.com/github/piyushhhhhh30/NITJ/blob/main/domain_aware_question_answering_system_using_rag_and_fine_tuned_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers datasets langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Example content
text = """
Digital banking allows users to transfer funds online via NEFT or RTGS.
Transactions above ₹2 lakh require OTP and two-step authentication.
Failed transactions may be reversed within 2–3 business days.
"""

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
documents = splitter.create_documents([text])
docs = [doc.page_content for doc in documents]

print(docs)


['Digital banking allows users to transfer funds online via NEFT or RTGS.\nTransactions above ₹2 lakh require OTP and two-step authentication.', 'Failed transactions may be reversed within 2–3 business days.']


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(docs)

index = faiss.IndexFlatL2(doc_embeddings[0].shape[0])
index.add(np.array(doc_embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "How do I reverse a failed transaction?"
query_vec = embedder.encode([query])
_, retrieved_idx = index.search(np.array(query_vec), k=2)

retrieved_docs = [docs[i] for i in retrieved_idx[0]]
print("\n".join(retrieved_docs))


Failed transactions may be reversed within 2–3 business days.
Digital banking allows users to transfer funds online via NEFT or RTGS.
Transactions above ₹2 lakh require OTP and two-step authentication.


In [ ]:
from transformers import pipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

context = " ".join(retrieved_docs)
prompt = f"Answer based on context:\n{context}\n\nQuestion: {query}"

response = qa_model(prompt, max_length=100)[0]['generated_text']
print("Answer:", response)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Answer: NEFT or RTGS


In [ ]:
from transformers import pipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

context = " ".join(retrieved_docs)
prompt = f"Answer based on context:\n{context}\n\nQuestion: {query}"

response = qa_model(prompt, max_length=100)[0]['generated_text']
print("Answer:", response)


Device set to use cpu


Answer: NEFT or RTGS


In [ ]:
from datasets import load_dataset
from datasets import load_dataset

!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json

import json

with open("train-v1.1.json", "r") as f:
    squad_data = json.load(f)

sample = squad_data["data"][0]["paragraphs"][0]
context = sample["context"]
question = sample["qas"][0]["question"]
answer = sample["qas"][0]["answers"][0]["text"]

print("Context:", context[:200], "...")
print("Question:", question)
print("Answer:", answer)


--2025-05-15 14:50:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M   169MB/s    in 0.2s    

2025-05-15 14:50:57 (169 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper sta ...
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer: Saint Bernadette Soubirous


In [ ]:
from sklearn.metrics import f1_score

y_true = ["Failed transactions may be reversed within 2–3 business days."]
y_pred = [response]

# Use string overlap or ROUGE/F1 scoring libraries


In [ ]:
import pickle

with open("faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)


In [ ]:
import pickle

with open("faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)


In [ ]:
!pip install gradio
import gradio as gr

def answer_question(q):
    q_vec = embedder.encode([q])
    _, idx = index.search(np.array(q_vec), k=2)
    context = " ".join([docs[i] for i in idx[0]])
    prompt = f"Answer based on: {context}\n\nQuestion: {q}"
    return qa_model(prompt)[0]['generated_text']

gr.Interface(fn=answer_question, inputs="text", outputs="text").launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a7394317a5d96e8d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory 

In [ ]:
from google.colab import files
import nbformat

# Upload your corrupted notebook
uploaded = files.upload()

for fname in uploaded:
    # Load notebook
    nb = nbformat.read(fname, as_version=4)

    # Delete broken widget metadata
    if 'widgets' in nb['metadata']:
        print(f"Cleaning metadata in: {fname}")
        del nb['metadata']['widgets']

    # Save cleaned notebook
    cleaned_fname = "cleaned_" + fname
    with open(cleaned_fname, 'w') as f:
        nbformat.write(nb, f)

    files.download(cleaned_fname)


Saving Domain_Aware_Question_Answering_System_using_RAG_and_Fine_Tuned_Transformers.ipynb to Domain_Aware_Question_Answering_System_using_RAG_and_Fine_Tuned_Transformers.ipynb
Cleaning metadata in: Domain_Aware_Question_Answering_System_using_RAG_and_Fine_Tuned_Transformers.ipynb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>